In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv

In [2]:
# Load Environment Variables
load_dotenv()

True

In [3]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type="sqlite"

aengine = create_async_engine(
    url=f"{db_type}+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [4]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
validation_llm = "gpt-4o"
guardrails_llm = "gpt-4o"
sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."
db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"
num_common_values = 10
indicator_vars="district_name" # This should be a comma delimited string in multiple vars


### Single-turn question

In [5]:
# Your question
query = {
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
}


In [6]:
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        query,
        session,
        metric_db_id,
        db_type,
        llm,
        guardrails_llm,
        sys_message,
        db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()
    print(qp.final_answer)

/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1152}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In Chennai, there are a total of 20,334 beds available. This total comes from two sources: 7,179 beds are from clinics, and 13,155 beds are from health centers and district hospitals. These numbers include beds with and without oxygen supply, as well as ICU beds.


### Multi-turn chat

In [10]:
# Simulating a conversation
queries = [{
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
},
{
    "query_text": "How about Ranipet??",
    "query_metadata": {}
},
{
    "query_text": "Yahan COVID patients ke baare me batao??",
    "query_metadata": {}
},
{
    "query_text": "Nahin, Ranipet ke liye manga tha",
    "query_metadata": {}
},
{
    "query_text": "How many beds with oxygen supply in Ariyalur??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients here?",
    "query_metadata": {}
},
{
    "query_text": "Nahin, Ranipet ke liye batao",
    "query_metadata": {}
},]

In [13]:
from askametric.query_processor.query_processor import MultiTurnQueryProcessor

chat_history = []
async with async_session() as session:
    for query in queries:
        mqp = MultiTurnQueryProcessor(
            query=query,
            asession=session,
            metric_db_id=metric_db_id,
            db_type=db_type,
            llm=llm,
            guardrails_llm=guardrails_llm,
            sys_message=sys_message,
            db_description=db_description,
            column_description="",
            indicator_vars=indicator_vars,
            num_common_values=num_common_values,
            chat_history=chat_history
        )

        await mqp.process_query()
        chat_history.append({"user": mqp.eng_translation["query_text"],
                             "system": mqp.translated_answer,})
        print(f"Q: {query['query_text']}")
        print(f"A: {mqp.final_answer}")
        print("\n")

Q: How many beds are there in chennai??
A: In Chennai, there are a total of 20,334 beds available. This total comes from two sources: 7,179 beds are from clinics, and 13,155 beds are from health centers and district hospitals. These numbers include beds with and without oxygen supply, as well as ICU beds.


Q: How about Ranipet??
A: The availability of beds for COVID-19 patients in Ranipet is as follows: There are 153 vacant beds in clinics and 90 vacant beds in health centers and district hospitals. This information was gathered by checking the current data on bed vacancies specifically for the Ranipet district.


Q: Yahan COVID patients ke baare me batao??
A: Tamil Nadu mein COVID-19 ke maamle kuch is prakar hain: Ariyalur mein 624 log ilaaj ke liye hain aur 106 naye maamle aaye hain. Chengalpattu mein 13038 log ilaaj ke liye hain aur 2419 naye maamle aaye hain. Chennai mein 37713 log ilaaj ke liye hain aur 7466 naye maamle aaye hain. Coimbatore mein 14324 log ilaaj ke liye hain aur 

/home/poornima/.cache/pypoetry/virtualenvs/askametric-JOXIZc-T-py3.11/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Q: Nahin, Ranipet ke liye manga tha
A: Ranipet ke liye COVID-19 ke maamle aur bed availability ki jaankari yeh hai: 11 May tak 2018 log ilaaj ke liye admit the, aur 11 May ko 423 naye maamle aaye. Us din 277 log discharge hue aur 2 logon ki maut hui. Abhi 2162 log ilaaj ke liye hospital mein hain. Bed availability ki baat karein toh, clinics mein 936 earmarked beds hain jismein se 783 occupied hain aur 153 vacant hain. Health centers aur district hospitals mein 265 earmarked beds with O2 supply hain, 339 without O2 supply, aur 29 ICU beds hain. Inmein se 265 beds with O2 supply, 249 without O2 supply, aur 29 ICU beds occupied hain. Vacant beds ki baat karein toh, O2 supply ke saath koi vacant bed nahi hai, bina O2 supply ke 90 vacant beds hain, aur ICU mein koi vacant bed nahi hai. Total 90 vacant beds hain.


Q: How many beds with oxygen supply in Ariyalur??
A: There are 314 beds with oxygen supply available in Ariyalur. This information was obtained from the database that tracks bed 